Now that the Model Training, Model Evaluation, Model Serving and Data Pipeline are in place, we have to connect all the parts together. The main aim of this Continous_X_pipeline is to go from ideation to actual model deployment quickly and have an established process to iterate. This is indeed the Ops in the MLOps!

We will be provisioning resources and installing tools via code. For this we would be using : 

-   Terraform: A declarative Infrastructure as Code (IaC) tool used to provision and manage cloud infrastructure (servers, networks, etc.) by defining the desired end state in configuration files. Here, we use it to provision our infrastructure.
-   Ansible: An imperative Configuration as Code (CaC) tool that automates system configuration, software installation, and application deployment through task-based YAML playbooks describing the steps to achieve a desired setup. Here, we use it to install Kubernetes and the Argo tools on our infrastructure after it is provisioned
-   Argo CD: A declarative GitOps continuous delivery tool for Kubernetes that automatically syncs and deploys applications based on the desired state stored in Git repositories.
-   Argo Workflows: A Kubernetes-native workflow engine where we define workflows, which execute tasks inside containers to run pipelines, jobs, or automation processes.

Let's get a copy of the Bird Classification Infrastructure repository

> **Note**: This is a Bash notebook, so you will run it with a Bash kernel. You can change the kernel (if needed) by clicking the kernel name in the top right of the Jupyter interface.

In [1]:
git clone --recurse-submodules https://github.com/exploring-curiosity/MLOps.git

Cloning into 'MLOps'...
remote: Enumerating objects: 2908, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 2908 (delta 70), reused 98 (delta 47), pack-reused 2782 (from 1)
Receiving objects: 100% (2908/2908), 22.74 MiB | 4.79 MiB/s, done.
Resolving deltas: 100% (680/680), done.
Updating files: 100% (4239/4239), done.


The code structure in continous_X_pipeline has has the following structure :

    ├── tf
    │   └── kvm
    ├── ansible
    │   ├── general
    │   ├── pre_k8s
    │   ├── k8s
    │   ├── post_k8s
    │   └── argocd
    ├── k8s
    │   ├── platform
    │   ├── staging
    │   ├── canary
    │   └── production
    └── workflows

### Install and configure Terraform

Before we can use Terraform, we’ll need to download a Terraform client. The following cell will download the Terraform client and “install” it in this environment:

In [2]:
mkdir -p /work/.local/bin
wget https://releases.hashicorp.com/terraform/1.10.5/terraform_1.10.5_linux_amd64.zip
unzip -o -q terraform_1.10.5_linux_amd64.zip
mv terraform /work/.local/bin
rm terraform_1.10.5_linux_amd64.zip

--2025-05-09 23:51:05--  https://releases.hashicorp.com/terraform/1.10.5/terraform_1.10.5_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 18.238.171.95, 18.238.171.54, 18.238.171.101, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|18.238.171.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27714924 (26M) [application/zip]
Saving to: ‘terraform_1.10.5_linux_amd64.zip’

terraform_1.10.5_li 100%[===================>]  26.43M   112MB/s    in 0.2s    

2025-05-09 23:51:06 (112 MB/s) - ‘terraform_1.10.5_linux_amd64.zip’ saved [27714924/27714924]



The Terraform client has been installed to: `/work/.local/bin`. In order to run `terraform` commands, we will have to add this directory to our `PATH`, which tells the system where to look for executable files.

In [3]:
export PATH=/work/.local/bin:$PATH

Let’s make sure we can now run `terraform` commands. The following cell should print usage information for the `terraform` command, since we run it without any subcommands:

In [4]:
terraform

Usage: terraform [global options] <subcommand> [args]

The available commands for execution are listed below.
The primary workflow commands are given first, followed by
less common or more advanced commands.

Main commands:
  init          Prepare your working directory for other commands
  validate      Check whether the configuration is valid
  plan          Show changes required by the current configuration
  apply         Create or update infrastructure
  destroy       Destroy previously-created infrastructure

All other commands:
  console       Try Terraform expressions at an interactive command prompt
  fmt           Reformat your configuration in the standard style
  force-unlock  Release a stuck lock on the current workspace
  get           Install or upgrade remote Terraform modules
  graph         Generate a Graphviz graph of the steps in an operation
  import        Associate existing infrastructure with a Terraform resource
  login         Obtain and save credentials for a

: 127

### Configure the PATH

Both Terraform and Ansible executables have been installed to a location that is not the system-wide location for executable files: `/work/.local/bin`. In order to run `terraform` or `ansible-playbook` commands, we will have to add this directory to our `PATH`, which tells the system where to look for executable files.

In [5]:
# runs in Chameleon Jupyter environment
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local

### Prepare Kubespray

To install Kubernetes, we’ll use Kubespray, which is a set of Ansible playbooks for deploying Kubernetes. We’ll also make sure we have its dependencies now:

In [6]:
PYTHONUSERBASE=/work/.local pip install --user -r MLOps/continous_X_pipeline/ansible/k8s/kubespray/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 12.5 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 6.8 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.2 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 19.3 MB/s eta 0:00:00



We will need to prepare credentials with which it can act on our behalf on the Chameleon OpenStack cloud. This is a one-time procedure.

To get credentials, open the Horizon GUI:

-   from the Chameleon website
-   click “Experiment” \> “KVM@TACC”
-   log in if prompted to do so
-   check the project drop-down menu near the top left (which shows e.g. “CHI-XXXXXX”), and make sure the correct project is selected.

On the left side, expand the “Identity” section and click on “Application Credentials”. Then, click “Create Application Credential”.

-   In the “Name”, field, use “mlops-lab”.
-   Set the “Expiration” date.
-   Click “Create Application Credential”.
-   Choose “Download clouds.yaml”.

Let's add the actual application_credential_id and application_credential_secret from the downloaded clouds.yaml in the clouds.yaml file that we see here. 

Terraform will look for the `clouds.yaml` in either `~/.config/openstack` or the directory from which we run `terraform` - we will move it to the latter directory:

In [ ]:
cp clouds.yaml /work/gourmetgram-iac/tf/kvm/clouds.yaml